In [ ]:
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import os
import cv2
import joblib
from sklearn.metrics import f1_score

## Функция для отображения маски

In [ ]:
def show_mask(mask):
    """Отображает маску с помощью matplotlib."""
    plt.figure(figsize=(10, 10))
    plt.imshow(mask, cmap='gray')
    plt.title("Предсказанная водная маска")
    plt.axis('off')
    plt.show()

## Используем сглаживание по Гауссу для размытия и устранения лагов модели

In [ ]:
def gaussian_smooth_mask(mask, kernel_size=(5, 5)):
    # Применяем фильтр Гаусса для размытия
    smoothed_mask = cv2.GaussianBlur(mask.astype(np.float32), kernel_size, 0)
    smoothed_mask = (smoothed_mask > 0.95).astype(np.uint8)  # Бинаризуем после размытия
    
    return smoothed_mask

## Функции для обработки изображения и создания маски при помощи индексов, расчитанных из каналов изображений

In [ ]:
def create_water_mask(indices):
    # Условия для каждого индекса
    ndwi_mask = indices['NDWI'] < 1   #74
    ndmi_mask = indices['NDMI'] < 1   #50
    mndwi_mask = indices['MNDWI'] < 1   #70
    wri_mask = indices['WRI'] > 1      #70
    ndvi_mask = indices['NDVI'] > 1    #70
    awei_mask = indices['AWEI'] < 0   #62

    # Объединяем условия для создания водной маски
    water_mask = (ndwi_mask | mndwi_mask | wri_mask | ndvi_mask).astype(np.uint8)
    return water_mask

# Функции для расчета водных индексов
def calculate_indices(blue, green, red, nir, mir, swir):
    indices = {
        'NDWI': (green - nir) / (green + nir),
        'NDMI': (nir - mir) / (nir + mir),
        'MNDWI': (green - mir) / (green + mir),
        'WRI': (green + red) / (nir + mir),
        'NDVI': (nir - red) / (nir + red),
        'AWEI': 4 * (green - mir) - (0.25 * nir + 2.75 * swir)
    }
    return indices

# Функции для нормализации и увеличения яркости
def normalize(band):
    band_min, band_max = band.min(), band.max()
    return (band - band_min) / (band_max - band_min)

# Функция для увеличения яркости спектральной полосы
def brighten(band):
    alpha = 0.13
    beta = 0
    return np.clip(alpha * band + beta, 0, 255)

# Функция для преобразования изображения
def convert(im_path):
    with rasterio.open(im_path) as fin:
        red = fin.read(3)
        green = fin.read(2)
        blue = fin.read(1)

    red_b = brighten(red)
    green_b = brighten(green)
    blue_b = brighten(blue)

    red_bn = normalize(red_b)
    green_bn = normalize(green_b)
    blue_bn = normalize(blue_b)

    return np.dstack((blue_b, green_b, red_b)), np.dstack((red_bn, green_bn, blue_bn))

# Основная функция для обработки изображения Sentinel-2A и создания масок
def process_image(image_path):
    with rasterio.open(image_path) as src:
        # Считывание необходимых каналов с понижением разрешения до 20 м
        blue = src.read(1)
        green = src.read(2)
        red = src.read(3)
        nir = src.read(7)
        mir = src.read(9)
        swir = src.read(10)

        # Расчет индексов
        indices = calculate_indices(blue, green, red, nir, mir, swir)

        # Создание водной маски
        water_mask = create_water_mask(indices)
    
    return water_mask

## Обработка изображения и создание маски при помощи CatBoost

In [ ]:
def load_image_as_dataframe_cat(image_path):
    """Загружает TIFF-изображение и преобразует его в DataFrame с рассчитанными водными индексами."""
    with rasterio.open(image_path) as img:
        image_data = img.read()  # Чтение всех каналов изображения
        image_data = image_data.reshape(image_data.shape[0], -1).T  # Преобразуем в формат (кол-во пикселей, кол-во каналов)
    
    # Создаем DataFrame из данных изображения
    df = pd.DataFrame(image_data, columns=[f'{i}' for i in range(image_data.shape[1])])

    # Вычисляем водные индексы
    df['NDWI'] = (df['1'] - df['6']) / (df['1'] + df['6'])  # green - nir
    df['NDMI'] = (df['6'] - df['8']) / (df['6'] + df['8'])  # nir - mir
    df['MNDWI'] = (df['1'] - df['8']) / (df['1'] + df['8'])  # green - mir
    df['WRI'] = (df['1'] + df['2']) / (df['6'] + df['8'])  # green + red / nir + mir
    df['NDVI'] = (df['6'] - df['2']) / (df['6'] + df['2'])  # nir - red / nir + red
    df['AWEI'] = 4 * (df['1'] - df['8']) - (0.25 * df['6'] + 2.75 * df['9'])  # 4*(green - mir) - (0.25 * nir + 2.75 * swir)

    return df

def predict_water_pixels_cat(image_path, model):
    """Загружает изображение, предсказывает водные пиксели и возвращает бинарную матрицу."""
    df = load_image_as_dataframe_cat(image_path)
    
    # Предсказание класса "вода" или "не вода"
    y_pred = model.predict(df)
    
    # Преобразование предсказаний в исходное разрешение изображения
    with rasterio.open(image_path) as img:
        height, width = img.height, img.width
    binary_mask = y_pred.reshape((height, width)).astype(np.uint8)  # Приводим к uint8 для бинарного изображения

    return binary_mask

## Создание маски и подгрузка изображения при помощи Логистической регрессис

In [ ]:
def load_image_as_dataframe_log(image_path):
    """Загружает TIFF-изображение и преобразует его в DataFrame с рассчитанными водными индексами."""
    with rasterio.open(image_path) as img:
        image_data = img.read()  # Чтение всех каналов изображения
        image_data = image_data.reshape(image_data.shape[0], -1).T  # Преобразуем в формат (кол-во пикселей, кол-во каналов)
    
    # Создаем DataFrame из данных изображения
    df = pd.DataFrame(image_data, columns=[f'{i}' for i in range(image_data.shape[1])])

    # Вычисляем водные индексы
    df['NDWI'] = (df['1'] - df['6']) / (df['1'] + df['6'])  # green - nir
    df['NDMI'] = (df['6'] - df['8']) / (df['6'] + df['8'])  # nir - mir
    df['MNDWI'] = (df['1'] - df['8']) / (df['1'] + df['8'])  # green - mir
    df['WRI'] = (df['1'] + df['2']) / (df['6'] + df['8'])  # green + red / nir + mir
    df['NDVI'] = (df['6'] - df['2']) / (df['6'] + df['2'])  # nir - red / nir + red
    df['AWEI'] = 4 * (df['1'] - df['8']) - (0.25 * df['6'] + 2.75 * df['9'])  # 4*(green - mir) - (0.25 * nir + 2.75 * swir)

    return df

def predict_water_pixels_log(image_path, model):
    """Загружает изображение, предсказывает водные пиксели и возвращает бинарную матрицу."""
    df = load_image_as_dataframe_log(image_path)
    df.fillna(df.mean(), inplace=True)
    
    # Предсказание класса "вода" или "не вода"
    features = df[['NDWI', 'NDMI', 'MNDWI', 'WRI', 'NDVI', 'AWEI']]  # Используем только необходимые признаки
    y_pred = model.predict(features)
    
    # Преобразование предсказаний в исходное разрешение изображения
    with rasterio.open(image_path) as img:
        height, width = img.height, img.width
    binary_mask = y_pred.reshape((height, width)).astype(np.uint8)  # Приводим к uint8 для бинарного изображения

    binary_mask = np.where(binary_mask == 1, 0, 1).astype(np.uint8)
    
    return binary_mask

## Подгрузка моделей CatBoost и Логичстической регрессии

In [ ]:
# Загрузка обученной модели CatBoost
model_path = 'catboost_model.cbm'  # Укажите путь к предварительно обученной модели
model_cat = CatBoostClassifier()
model_cat.load_model(model_path)

In [ ]:
# Загрузка обученной модели логистической регрессии
model_path = 'logistic_regression_model.joblib'
model_log = joblib.load(model_path)

In [ ]:
image_path = 'data/train/images/5.tif'
output_mask_path = 'zmasks/5.tif'

In [ ]:
water_mask_index = process_image(image_path)
water_mask_cat = gaussian_smooth_mask(predict_water_pixels_cat(image_path, model_cat))
water_mask_log = gaussian_smooth_mask(predict_water_pixels_cat(image_path, model_cat))

In [ ]:
show_mask(water_mask_index)
show_mask(water_mask_cat)
show_mask(water_mask_log)

In [ ]:
def combine_masks(mask1, mask2, mask3):
    """Объединяет три бинарные маски в одну с помощью логического "ИЛИ"."""
    combined_mask = np.logical_or(mask1, mask2)
    combined_mask = np.logical_or(combined_mask, mask3).astype(np.uint8)
    return combined_mask

In [ ]:
water_mask_result = combine_masks(water_mask_index, water_mask_cat, water_mask_log)
show_mask(water_mask_result)

In [ ]:
def save_binary_mask(mask, reference_image_path, output_path):
    """Сохраняет бинарную маску в формате TIFF с тем же разрешением, что и оригинальное изображение."""
    # Открываем оригинальное изображение для извлечения метаданных
    with rasterio.open(reference_image_path) as src:
        meta = src.meta  # Метаданные исходного изображения
        meta.update({
            'count': 1,       # Один канал для маски
            'dtype': 'uint8', # Тип данных для бинарного изображения
            'compress': 'lzw' # Опционально, добавляем сжатие LZW
        })

        # Сохраняем маску как изображение .tif
        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(mask, 1)  # Записываем в первый канал маску

In [ ]:
save_binary_mask(water_mask_result, image_path, output_mask_path)
print(f"Бинарная маска успешно сохранена по пути: {output_mask_path}")

In [ ]:
def load_raster(file_path):
    """Load a raster image and flatten it into a 1D array."""
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the first band
    return data.flatten()

def calculate_f1_score(pred_path, gt_path):
    """Calculate the F1 score between the predicted and ground truth masks."""
    # Load the images
    pred_mask = load_raster(pred_path)
    gt_mask = load_raster(gt_path)

    # Calculate F1 score
    f1 = f1_score(gt_mask, pred_mask, average='macro')
    return f1

def main():
    # Пути к предсказанным и эталонным маскам
    predicted_masks = [
        r'zmasks\5.tif'
    ]
    
    ground_truth_masks = [
        r'data\train\masks\5.tif'
    ]

    # Проверка, что количество масок совпадает
    if len(predicted_masks) != len(ground_truth_masks):
        print("Количество предсказанных масок и эталонных масок должно совпадать.")
        return

    # Сравнение масок и вывод F1-оценок
    for pred_mask, gt_mask in zip(predicted_masks, ground_truth_masks):
        f1 = calculate_f1_score(pred_mask, gt_mask)
        print(f'F1 Score for {os.path.basename(pred_mask)}: {f1:.3f}')

if __name__ == "__main__":
    main()
